In [1]:
import pandas as pd

df = pd.read_csv('D1BaseballTrackman.csv')


In [3]:
sec_teams = [
    "TEX_AGG", "ARK_RAZ", "LSU_TIG", "OLE_REB", "MSU_BDG",
    "ALA_CRI", "AUB_TIG", "VAN_COM", "TEN_VOL", "GEO_BUL",
    "SOU_GAM", "FLA_GAT", "KEN_WIL", "MIZ_TIG"
]
df['Events'] = df['Events'].replace('HitbyPitch', 'HitByPitch')
df['Events'] = df['Events'].replace('Hitbypitch', 'HitByPitch')
df['Events'] = df['Events'].replace('StirkeCalled', 'StrikeCalled')

# Filter the dataframe for the specific team abbreviations in both BatterTeam and PitcherTeam
df['SEC_Game'] = df.apply(lambda row: row['PitcherTeam'] in sec_teams and row['BatterTeam'] in sec_teams, axis=1)
dfs = df[df['SEC_Game']]

In [3]:
#sample_df = data.sample(n=100)
#sample_df.to_csv('sample_data.csv', index=False)


In [19]:
df['Events'].unique()
dfs['AtBatID'] = dfs['GameID'].astype(str) + '_' + dfs['Batter'].astype(str) + '_' + dfs['Inning'].astype(str) + '_' + dfs['Pitcher'].astype(str)
df_0_0 = dfs[dfs['Count'] == '0-0']


array(['Single', 'Out', 'FoulBall', 'StrikeCalled', 'StrikeSwinging',
       'BallCalled', 'BallinDirt', 'HomeRun', 'Double', 'HitByPitch',
       'Triple', 'FieldersChoice'], dtype=object)

In [31]:

# Sort the DataFrame to ensure it's in chronological order, which is crucial for accurately identifying the first pitch and the outcome of each at-bat
df_sorted = dfs.sort_values(['Date', 'GameID', 'Inning', 'AtBatID'])

# Identify the first pitch of each at-bat as the row where 'Count' is '0-0'
df_sorted['IsFirstPitch'] = df_sorted.groupby('AtBatID')['Count'].transform(lambda x: x == '0-0')

# Only keep the first occurrence of '0-0' count as the first pitch for each at-bat
df_first_pitch = df_sorted[df_sorted['IsFirstPitch']].drop_duplicates(subset=['AtBatID'])

df_first_pitch['FirstPitchOutcome'] = df_first_pitch['Events'].apply(
    lambda x: 'Strike' if x in ['StrikeCalled', 'StrikeSwinging', 'FoulBall'] else ('Ball' if x in ['BallCalled', 'HitByPitch', 'BallinDirt'] else 'InPlay')
)

# Continue with the previously outlined steps to merge this information back into the main DataFrame
df_merged = df_sorted.merge(df_first_pitch[['AtBatID', 'FirstPitchOutcome']], on='AtBatID', how='left')

# Now, update the logic to identify at-bats ending in an out based on this refined outcome categorization
df_merged['AtBatEndsInOut'] = df_merged['Events'] == 'Out'
df_merged['AtBatEndsInOut'] = df_merged.groupby('AtBatID')['AtBatEndsInOut'].transform('max')

# Calculate the total at-bats for each first pitch outcome and the number ending in an out
df_at_bat = df_merged.groupby('AtBatID').agg({
    'FirstPitchOutcome': 'first',  # Since all rows per at-bat have the same outcome after merge, 'first' works
    'AtBatEndsInOut': 'max'  # 'max' here captures if any part of the at-bat ended in an out (True if any row is True)
}).reset_index()

# Calculate the number of at-bats NOT ending in an out for each first pitch outcome
at_bats_not_ending_in_out_strike = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'Strike') & (~df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()
at_bats_not_ending_in_out_ball = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'Ball') & (~df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()
at_bats_not_ending_in_out_inplay = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'InPlay') & (~df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()

# Total at-bats for each first pitch outcome (calculated previously)
total_at_bats_strike = df_at_bat[df_at_bat['FirstPitchOutcome'] == 'Strike']['AtBatID'].nunique()
total_at_bats_ball = df_at_bat[df_at_bat['FirstPitchOutcome'] == 'Ball']['AtBatID'].nunique()
total_at_bats_inplay = df_at_bat[df_at_bat['FirstPitchOutcome'] == 'InPlay']['AtBatID'].nunique()

# Now df_at_bat contains one row per at-bat with the necessary information
# Calculate the total number of at-bats for each first pitch outcome
total_at_bats = df_at_bat['AtBatID'].nunique()

# Calculate the number of at-bats ending in an out for each first pitch outcome
at_bats_ending_in_out_strike = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'Strike') & (df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()
at_bats_ending_in_out_ball = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'Ball') & (df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()
at_bats_ending_in_out_inplay = df_at_bat[(df_at_bat['FirstPitchOutcome'] == 'InPlay') & (df_at_bat['AtBatEndsInOut'])]['AtBatID'].nunique()

# Calculate probabilities
prob_out_after_strike = at_bats_ending_in_out_strike / total_at_bats_strike 
prob_out_after_ball = at_bats_ending_in_out_ball / total_at_bats_ball


prob_out_after_strike, prob_out_after_ball


C:\Users\kearn\AppData\Local\Temp/ipykernel_17192/200192304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['AtBatID'] = dfs['GameID'].astype(str) + '_' + dfs['Batter'].astype(str) + '_' + dfs['Inning'].astype(str) + '_' + dfs['Pitcher'].astype(str)


(0.3485576923076923, 0.3375887493173129)

In [32]:
chance_no_out_after_strike = at_bats_not_ending_in_out_strike / total_at_bats_strike
chance_no_out_after_ball = at_bats_not_ending_in_out_ball / total_at_bats_ball

chance_no_out_after_strike, chance_no_out_after_ball


(0.6514423076923077, 0.662411250682687)

In [28]:

df_filtered = df_0_0[df_0_0['Events'] != 'BallCalled']
df_filtered = df_filtered[df_filtered['Events'] != 'HitByPitch']
df_filtered = df_filtered[df_filtered['Events'] != 'BallinDirt']
df_filtered = df_filtered[df_filtered['Events'] != 'FieldersChoice']

# Step 3: Calculate the probability where the event is either 'Out' or a type of strike
# Define the types of strikes for clarity
strike_types = ['StrikeCalled', 'StrikeSwinging', 'FoulBall']

# Count the number of rows where the event is 'Out' or a type of strike
num_out_or_strike = df_filtered[df_filtered['Events'].isin(strike_types + ['Out'])].shape[0]

num_hits = df_filtered[df_filtered['Events'].isin(['Single', 'Double', 'Triple', 'HomeRun'])].shape[0]

# Total number of rows after filtering (Step 2)
total_rows_after_filtering = df_filtered.shape[0]

# Calculate the probability
probability_out_or_strike = num_out_or_strike / total_rows_after_filtering if total_rows_after_filtering > 0 else 0

probability_hit = num_hits / total_rows_after_filtering if total_rows_after_filtering > 0 else 0

probability_out_or_strike, probability_hit 

In [34]:
counts = ['0-0', '0-1', '0-2', '1-2', '2-2', '3-2']

# Define the events to exclude
excluded_events = ['BallCalled', 'HitByPitch', 'BallinDirt', 'FieldersChoice']

# Define the types of strikes for clarity
strike_types = ['StrikeCalled', 'StrikeSwinging', 'FoulBall']

# Initialize an empty list to store the results
results = []

for count in counts:
    # Filter the DataFrame for the current count and exclude specific events
    df_filtered = dfs[(dfs['Count'] == count) & (~dfs['Events'].isin(excluded_events))]
    
    # Count the number of rows where the event is 'Out' or a type of strike
    num_out_or_strike = df_filtered[df_filtered['Events'].isin(strike_types + ['Out'])].shape[0]
    
    # Count the number of hits
    num_hits = df_filtered[df_filtered['Events'].isin(['Single', 'Double', 'Triple', 'HomeRun'])].shape[0]
    
    # Total number of rows after filtering
    total_rows_after_filtering = df_filtered.shape[0]
    
    # Calculate the probabilities
    probability_out_or_strike = num_out_or_strike / total_rows_after_filtering if total_rows_after_filtering > 0 else 0
    probability_hit = num_hits / total_rows_after_filtering if total_rows_after_filtering > 0 else 0
    
    # Append the results
    results.append({
        'Count': count,
        'Probability Out or Strike': probability_out_or_strike,
        'Probability Hit': probability_hit
    })

# Convert the results into a DataFrame for display
results_df = pd.DataFrame(results)

# Display the table
print(results_df)

  Count  Probability Out or Strike  Probability Hit
0   0-0                   0.934273         0.065727
1   0-1                   0.904190         0.095810
2   0-2                   0.888889         0.111111
3   1-2                   0.885959         0.114041
4   2-2                   0.879493         0.120507
5   3-2                   0.868083         0.131917


In [27]:
total_rows_after_filtering 

20083

In [33]:
df_0_0['Events'].unique()

array(['FoulBall', 'BallCalled', 'StrikeCalled', 'Single', 'Out',
       'StrikeSwinging', 'Double', 'HomeRun', 'HitByPitch', 'BallinDirt',
       'FieldersChoice', 'Triple'], dtype=object)

In [35]:
num_specified_events = df_0_0[df_0_0['Events'].isin(['BallCalled', 'HitByPitch', 'BallinDirt'])].shape[0]

# Get the total row count of the DataFrame
total_rows = df_0_0.shape[0]

# Calculate the proportion of specified events
firstpitch_ball_percentage = num_specified_events / total_rows if total_rows > 0 else 0

firstpitch_ball_percentage

0.42195999424377606

In [36]:
first_pitch_strike_percentage = 1 - firstpitch_ball_percentage

first_pitch_strike_percentage

0.5780400057562239

In [38]:
df_0_0['PitchCall'].unique()

array(['FoulBall', 'BallCalled', 'StrikeCalled', 'InPlay',
       'StrikeSwinging', 'HitByPitch', 'BallinDirt'], dtype=object)

In [42]:
df['KorBB'].unique()

array(['Undefined', 'Walk', 'Strikeout'], dtype=object)

In [44]:
df_sorted = dfs.sort_values(by=['GameID', 'Inning', 'Batter', 'Date'], ascending=True)

# Identify the first pitch of each at-bat
df_sorted['IsFirstPitch'] = df_sorted.groupby(['GameID', 'Inning', 'Batter'])['Count'].transform(lambda x: x == '0-0')

# Filter for rows that are the first pitch of an at-bat
first_pitches = df_sorted[df_sorted['IsFirstPitch']]

# Determine whether the first pitch is a strike or a ball
first_pitches['FirstPitchType'] = first_pitches['Events'].apply(lambda x: 'Strike' if x in ['FoulBall', 'StrikeCalled', 'StrikeSwinging']
                                                                else 'Ball' if x in ['BallCalled', 'BallinDirt']
                                                                else 'Unknown')

# Now, filter the original DataFrame for strikeouts and walks
strikeouts = df_sorted[df_sorted['KorBB'] == 'Strikeout']
walks = df_sorted[df_sorted['KorBB'] == 'Walk']

# Merge the strikeouts and walks with the first_pitches to associate the first pitch type with each event
strikeouts_with_first_pitch = pd.merge(strikeouts, first_pitches[['GameID', 'Inning', 'Batter', 'FirstPitchType']], 
                                       on=['GameID', 'Inning', 'Batter'], 
                                       how='left')

walks_with_first_pitch = pd.merge(walks, first_pitches[['GameID', 'Inning', 'Batter', 'FirstPitchType']], 
                                  on=['GameID', 'Inning', 'Batter'], 
                                  how='left')

# Calculate the percentage of strikeouts and walks that start with a first pitch strike or ball
percentage_strikeouts_first_pitch_strike = round((strikeouts_with_first_pitch['FirstPitchType'] == 'Strike').mean() * 100, 2)
percentage_walks_first_pitch_ball = round((walks_with_first_pitch['FirstPitchType'] == 'Ball').mean() * 100, 2)

print(f"Percentage of strikeouts that start with a first pitch strike: {percentage_strikeouts_first_pitch_strike}%")
print(f"Percentage of walks that start with a first pitch ball: {percentage_walks_first_pitch_ball}%")

C:\Users\kearn\AppData\Local\Temp/ipykernel_17192/484697343.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_pitches['FirstPitchType'] = first_pitches['Events'].apply(lambda x: 'Strike' if x in ['FoulBall', 'StrikeCalled', 'StrikeSwinging']


Percentage of strikeouts that start with a first pitch strike: 65.85%
Percentage of walks that start with a first pitch ball: 71.9%


In [46]:
df_3_0 = dfs[dfs['Count'] == '3-0']

num_specified_events = df_3_0[df_3_0['Events'].isin(['BallCalled', 'HitByPitch', 'BallinDirt'])].shape[0]

# Get the total row count of the DataFrame
total_rows = df_3_0.shape[0]

# Calculate the proportion of specified events
firstpitch_ball_percentage = num_specified_events / total_rows if total_rows > 0 else 0

firstpitch_ball_percentage

0.40827147401908803

In [47]:
first_pitch_strike_percentage = 1 - firstpitch_ball_percentage

first_pitch_strike_percentage

0.591728525980912